In [6]:
import pandas as pd

A = pd.read_csv("A.csv").values
import numpy as np

x = np.array(A[:, 0:3].astype(np.float32))
x = np.hstack([x, np.ones([len(x[:, 0]), 1])])

y = np.array(A[:, 3:5].astype(np.float32))
from homography import Transformer

t = Transformer(img_size=512)
t.fit_homography(x, y)
import pickle

with open('./transformer.pickle', 'wb') as f:
    pickle.dump(t, f)

Homography Residuals: [0.00242039]
Least Squares Check: b, A @ X, b - A @ X
[[ 1.89453125e-01  1.85412303e-01  4.04082185e-03]
 [ 1.03515625e-01  9.34743404e-02  1.00412846e-02]
 [ 4.60937500e-01  4.49132447e-01  1.18050527e-02]
 [ 4.29687500e-01  4.21947030e-01  7.74047024e-03]
 [ 4.16015625e-01  4.10748497e-01  5.26712799e-03]
 [ 6.26953125e-01  6.10679390e-01  1.62737352e-02]
 [ 8.33984375e-01  8.38516184e-01 -4.53180865e-03]
 [ 8.78906250e-02  9.37170243e-02 -5.82639928e-03]
 [ 4.55078125e-01  4.59626799e-01 -4.54867440e-03]
 [ 8.32031250e-01  8.57598957e-01 -2.55677070e-02]
 [ 4.45312500e-01  4.44487474e-01  8.25025896e-04]
 [ 5.17578125e-01  5.06008184e-01  1.15699411e-02]
 [ 5.21484375e-01  5.24569517e-01 -3.08514181e-03]
 [ 8.32031250e-01  8.15979829e-01  1.60514213e-02]
 [ 4.70703125e-01  4.67775204e-01  2.92792143e-03]
 [ 4.35546875e-01  4.45569965e-01 -1.00230898e-02]
 [ 2.20703125e-01  2.33403450e-01 -1.27003250e-02]
 [ 2.57812500e-01  2.78072156e-01 -2.02596563e-02]]
